In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
tf.random.set_seed(1234)

In [3]:
import pandas as pd

In [4]:
import glob

folder_path = "/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling"  # Replace with the actual path to your folder

# Use glob to get a list of all files in the folder
files = glob.glob(folder_path + "/*")

# Now, the 'files' variable contains a list of file paths in the specified folder
print(files)


['/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling13.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling5.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling12.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling16.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling14.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling6.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling4.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling11.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling2.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling10.csv', '/content/drive/MyDrive/lidar_fall_detectio

In [5]:
def parse_dataset(num_points=400):

    train_points = []
    train_labels = []
    class_map = {}
    folder_path = "/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main"
    folders = glob.glob(folder_path + "/*")

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = folder.split("/")[-1]
        # gather all files

        train_files = glob.glob(os.path.join(folder_path,folder)+"/*")

        for f in train_files:


            # Use the random indices to select 200 random rows from the original array
            train_points.append(pd.read_csv(f).set_index('Unnamed: 0').sample(num_points).values)
            train_labels.append(i)

    return (
        np.array(train_points),
        np.array(train_labels),
        class_map,
    )

In [6]:
train_points = []
train_labels = []
class_map = {}
folder_path = "/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main"
folders = glob.glob(folder_path + "/*")

for i, folder in enumerate(folders):
  print("processing class: {}".format(os.path.basename(folder)))
  # store folder name with ID so we can retrieve later
  class_map[i] = folder.split("/")[-1]
  # gather all files

  train_files = glob.glob(os.path.join(folder_path,folder)+"/*")

  for f in train_files:
    train_points.append(pd.read_csv(f).set_index('Unnamed: 0'))
    train_labels.append(i)

processing class: falling
processing class: lying_down
processing class: sitting
processing class: standing


In [7]:
train_points[0].shape

(698, 3)

In [8]:
NUM_POINTS = 260
NUM_CLASSES = 4
BATCH_SIZE = 32
train_points,train_labels,class_map = parse_dataset(NUM_POINTS)

processing class: falling
processing class: lying_down
processing class: sitting
processing class: standing


In [9]:
def augment(points, label):
    # jitter points
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    # shuffle points
    points = tf.random.shuffle(points)
    return points, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)


In [10]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

In [11]:
class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

In [12]:

def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [13]:
inputs = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
model.summary()

Model: "pointnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 260, 3)]             0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 260, 32)              128       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 260, 32)              128       ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 260, 32)              0         ['batch_normalization[0

In [14]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)

model.fit(train_dataset, epochs=20)

Epoch 1/20
10/10 [==============================] - 11s 259ms/step - loss: 3.0240 - sparse_categorical_accuracy: 0.5845
Epoch 2/20
10/10 [==============================] - 3s 299ms/step - loss: 1.9114 - sparse_categorical_accuracy: 0.8446
Epoch 3/20
10/10 [==============================] - 2s 238ms/step - loss: 1.6605 - sparse_categorical_accuracy: 0.8412
Epoch 4/20
10/10 [==============================] - 2s 240ms/step - loss: 1.5846 - sparse_categorical_accuracy: 0.8615
Epoch 5/20
10/10 [==============================] - 3s 286ms/step - loss: 1.5809 - sparse_categorical_accuracy: 0.8716
Epoch 6/20
10/10 [==============================] - 3s 268ms/step - loss: 1.4876 - sparse_categorical_accuracy: 0.8885
Epoch 7/20
10/10 [==============================] - 3s 289ms/step - loss: 1.5392 - sparse_categorical_accuracy: 0.8716
Epoch 8/20
10/10 [==============================] - 3s 262ms/step - loss: 1.5112 - sparse_categorical_accuracy: 0.8716
Epoch 9/20
10/10 [=============================

In [15]:
y = model.predict(np.expand_dims(train_points[20],axis=0))

1/1 [==============================] - 0s 456ms/step


In [16]:
y

array([[0.04768863, 0.6276681 , 0.27108386, 0.05355945]], dtype=float32)